In [1]:
import pickle
import json
from glob import glob
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import h5py

# Add the mind-vis code folder to path
sys.path.append(r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\code\Mind_Vis_utils")

# mind-vis imports
#from sc_mbm.mae_for_fmri import MAEforFMRI, fmri_encoder
from Mind_Vis_utils.dc_ldm.ldm_for_fmri import cond_stage_model
from Mind_Vis_utils.dc_ldm.ldm_for_fmri import create_model_from_config, fLDM
from Mind_Vis_utils.dataset import create_Kamitani_dataset, create_BOLD5000_dataset

In [36]:
# Imagenet dataset

path_mindvis_imagenet_labels = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\data\BOLD5000\BOLD5000_Stimuli\Image_Labels\imagenet_final_labels.txt"
path_imagenet_captions = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\ImageNet-captions\imagenet_captions.json"
path_mindvis_bold_imagenet_files = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\data\BOLD5000\BOLD5000_Stimuli\Scene_Stimuli\Presented_Stimuli\ImageNet\*"
path_mindvis_kamitani_imagenet_files = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\data\Kamitani\npz\imagenet_testing_label.csv"

with open(path_mindvis_imagenet_labels, 'rb') as f:
          mindvis_imagenet_labels = f.read()
        

mindvis_imagenet_labels = mindvis_imagenet_labels.splitlines()
wnid_list = []
for line in mindvis_imagenet_labels:
    wnid_list.append(line.split()[0])
    
files_list = []
bold_imagenet_list = glob(path_mindvis_bold_imagenet_files)
bold_imagenet_list = [os.path.split(img)[1] for img in bold_imagenet_list]

kamitani_imagenet_list = pd.read_csv(path_mindvis_kamitani_imagenet_files, sep=',', header=None, names=['a', 'filename'])
kamitani_imagenet_list = kamitani_imagenet_list.filename

counts = 0
with open(path_imagenet_captions) as f:
    imnet_cap_json = json.load(f)
    for img in imnet_cap_json:
        if (img['filename'] in bold_imagenet_list) and (not img['description']):
            counts += 1
            
print(f"Number of imagenet images with descriptions in Bold: {counts} out of {len(bold_imagenet_list)}")

counts = 0
with open(path_imagenet_captions) as f:
    imnet_cap_json = json.load(f)
    for img in imnet_cap_json:
        if (img['filename'] in kamitani_imagenet_list.values) and (not img['description']):
            counts += 1
            
print(f"Number of imagenet images with descriptions in Kamitani: {counts} out of {len(kamitani_imagenet_list)}")

Number of imagenet images with descriptions in Bold: 152 out of 1916
Number of imagenet images with descriptions in Kamitani: 6 out of 50


In [35]:
'n01443537_22563.JPEG' in kamitani_imagenet_list.values

True

In [ ]:
# COCO dataset

path_coco_captions = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\COCO-captions\annotations\captions_train2014.json"
path_coco_images = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\data\BOLD5000\BOLD5000_Stimuli\Scene_Stimuli\Presented_Stimuli\COCO"

with open(path_coco_captions, 'rb') as f:
    coco_caption_json = json.load(f)
    
coco_cap = pd.DataFrame.from_records(coco_caption_json['annotations'])
mindvis_coco_list = glob(f"{path_coco_images}\*")
mindvis_coco_list = [os.path.split(img)[1] for img in mindvis_coco_list]

coco_cap_unique_img_id = coco_cap.image_id.unique()
coco_cap_unique_img_id_as_filenames = [f"COCO_train2014_{img_id:012d}.jpg" for img_id in coco_cap_unique_img_id]

counts = 0
for mindvis_coco_img in mindvis_coco_list:
    if mindvis_coco_img in coco_cap_unique_img_id_as_filenames:
        counts += 1

print(f"Number of Coco images with descriptions: {counts} out of {len(mindvis_coco_list)}")

In [ ]:
path_mindvis_GOD_fmri_encoder = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\pretrains\GOD\fmri_encoder.pth"
path_mindvis_BOLD5000_fmri_encoder = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\pretrains\BOLD5000\fmri_encoder.pth"
path_mindvis_GOD_finetuned = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\pretrains\GOD\finetuned.pth"
mindvis_GOD_fmri_encoder_state_dict = torch.load(path_mindvis_GOD_fmri_encoder, map_location='cpu')
config = mindvis_GOD_fmri_encoder_state_dict['config']
num_voxels = (mindvis_GOD_fmri_encoder_state_dict['model']['pos_embed'].shape[1] - 1)* config.patch_size





In [ ]:
# mindvis cond_stage_model
brain_encoder = fmri_latent_encoder(mindvis_GOD_fmri_encoder_state_dict,num_voxels = 1024, global_pool=False)

# mindvis fmri_encoder
#brain_encoder = create_model_from_config(config, num_voxels, global_pool=True)
#brain_encoder.load_checkpoint(mindvis_GOD_fmri_encoder_state_dict['model'])

In [ ]:
fmri_sample = torch.Tensor(dataset_train[0]['fmri'])
brain_encoder.forward(fmri_sample)

In [ ]:
#list(mindvis_GOD_fmri_encoder_state_dict.items())[0
config_pretrain = mindvis_GOD_fmri_encoder_state_dict['config']

num_voxels = (mindvis_GOD_fmri_encoder_state_dict['model']['pos_embed'].shape[1] - 1)* config_pretrain.patch_size
brain_encoer_model = MAEforFMRI(num_voxels=num_voxels, patch_size=config_pretrain.patch_size, embed_dim=config_pretrain.embed_dim,
                                decoder_embed_dim=config_pretrain.decoder_embed_dim, depth=config_pretrain.depth, 
                                num_heads=config_pretrain.num_heads, decoder_num_heads=config_pretrain.decoder_num_heads, 
                                mlp_ratio=config_pretrain.mlp_ratio, focus_range=None, use_nature_img_loss=False) 
brain_encoer_model.load_state_dict(mindvis_GOD_fmri_encoder_state_dict['model'], strict=False)

In [ ]:
# Creating GOD Dataset
path_kamitani_GOD = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\data\Kamitani\npz"
dataset_train, dataset_test = create_Kamitani_dataset(path_kamitani_GOD)


In [ ]:
# mindvis fLDM
pretrain_mbm_metafile = torch.load(path_mindvis_GOD_finetuned, map_location='cpu')
path_gm_pretrain = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\pretrains\ldm\label2img"
num_of_voxels = dataset_train.num_voxels
generative_model = fLDM(pretrain_mbm_metafile, num_of_voxels, device='cpu', pretrain_root=path_gm_pretrain, global_pool=False)

In [ ]:
#### Loading cond_stage_model (fmri_latent_encoder) saved from script gen_eval_get_encoder
path_fmri_encoder_from_final_mindvis = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\pretrains\fmri_encoder_from_final_mindvis.pth"
final_encoder_state_dict = torch.load(path_fmri_encoder_from_final_mindvis, map_location='cpu')
saved_encoder = fmri_latent_encoder(final_encoder_state_dict['metafile'], 
                                    final_encoder_state_dict['num_voxels'], 
                                    final_encoder_state_dict['cond_dim'],
                                    final_encoder_state_dict['global_pool'])

print("Loading state dict for saved_encoder")
saved_encoder.load_state_dict(final_encoder_state_dict['model'], strict=False)

In [ ]:
fmri_sample = dataset_train.fmri[0]
#fmri_embed = saved_encoder.forward(torch.Tensor(hcp['V1']))

path_bold_roi = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\BOLD5000_ROI_from_internet\ROIs\CSI1\h5\CSI1_ROIs_TR3.h5"
with h5py.File(path_bold_roi, 'r') as f:
    print(f.keys())
    print(len(f['LHLOC']))
    print(f['LHPPA'][2].shape)




In [ ]:
path_hcp = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\data\HCP\npz\100206\HCP_visual_voxel.npz"
path_bold = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Vis-clone\data\BOLD5000\BOLD5000_GLMsingle_ROI_betas\py\CSI1_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_allses_LHEarlyVis.npy"

hcp = np.load(path_hcp)
bold = np.load(path_bold)

In [2]:
#### Working fmri encoder! ####
#### Loading cond_stage_model (fmri_latent_encoder) saved from script gen_eval_get_encoder
path_fmri_encoder_from_final_mindvis = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\code\Mind_Vis_utils\dc_ldm\pretrains\fmri_encoder_from_final_mindvis.pth"
final_encoder_state_dict = torch.load(path_fmri_encoder_from_final_mindvis, map_location='cpu')
saved_encoder = cond_stage_model(final_encoder_state_dict['metafile'], 
                                    final_encoder_state_dict['num_voxels'], 
                                    final_encoder_state_dict['cond_dim'],
                                    final_encoder_state_dict['global_pool'])

print("Loading state dict for saved_encoder")
saved_encoder.load_state_dict(final_encoder_state_dict['model'], strict=False)

# Creating GOD Dataset
path_kamitani_GOD = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\data\Kamitani\npz"
dataset_train, dataset_test = create_Kamitani_dataset(path_kamitani_GOD)
train_dataloader = DataLoader(dataset_train, batch_size=3)


sample = iter(train_dataloader)
s1 = sample.next()
s1_fmri_batch = s1['fmri'].float()
saved_encoder = saved_encoder.float()
train_embed = saved_encoder.forward(s1_fmri_batch)




Position interpolate from 262 to 291
missing keys: ['decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'decoder_blocks.1.norm2.bias', 'decoder_blocks.1.mlp.fc1.weight', 'decoder_blocks.1.mlp.fc1.bias', 'decoder_blocks.1.mlp.fc2.weight', 'decoder_blocks.1.mlp.fc2.bias', 'decoder_blocks.2.norm1.weight', 'decoder_blocks.2.norm1.bias', 'deco

In [18]:
#print(s1_fmri_batch.shape)
#print(train_embed.shape)
p = r"C:\Users\roeys\OneDrive - Technion\Semester 7\DL\Project\Mind-Cap\Mind-Cap\data\BOLD5000\BOLD5000_Stimuli\Scene_Stimuli\Presented_Stimuli\img_dict.npy"
img_dict = np.load(p, allow_pickle=True).item()

In [34]:
img_dict['COCO_train2014_000000000036.jpg'].shape



(256, 256, 3)